In [34]:
import pandas as pd
pd.set_option('display.max_rows', 500)

res = pd.read_csv("/Users/dmitriisid/Desktop/python/webscraping/notebooks/db_pomoci_flagged_new_V2.csv", dtype={'Telefon': str})
res.drop("web", axis = 1, inplace = True)
res.rename(columns={'Webová stránka': 'web'}, inplace=True)
res_unmatched = res[res["Matched"]!="matched"].sort_values(by="Matched")
res_unmatched.head()

,Název,Kategorie,Adresa,E-mail,Telefon,web,Matched,Source
530,Klinický psycholog - Nemocnice Třebíč,Psychologická ambulance,"Purkyňovo nám. 133/2, Jejkov 674 01, Kraj Vyso...",info@nem-tr.cz,+420568809370,nem-tr.cz/cze/ambulance/klinicky-psycholog/,unmatched,NaN
915,Odborné sociální poradenství,Odborné sociální poradenství,"Velká Krajská 47/7, Litoměřice-Město 412 01, Ú...",litomerice@fokuslabe.cz,+420413034461,http://www.fokuslabe.cz,unmatched,NaN
916,"Pedagogicko-psychologická poradna, České Buděj...",Pedagogicko-psychologické poradny,"Nerudova 859/59, České Budějovice 3, 370 04 Če...",poradna.info@pppcb.cz,+420723392134,https://www.pppcb.cz/,unmatched,NaN
917,"Pedagogicko-psychologická poradna, Plzeň, Část...",Pedagogicko-psychologické poradny,"Částkova 691/78, Lobzy, 326 00 Plzeň",poradnaplzen@pepor-plzen.cz,+420377468111,https://www.pepor-plzen.cz/,unmatched,NaN
918,Pedagogicko-psychologická poradna Ústeckého kr...,Pedagogicko-psychologické poradny,"Lípová 651/9, 415 01 Teplice",info@pppuk.cz,NaN,https://www.pppuk.cz/,unmatched,NaN


In [66]:
SCRAPED_DATA_PATH = '/Users/dmitriisid/Desktop/python/webscraping/data/2_06/df_scraped_2_06.csv'
result_scraper = pd.read_csv(SCRAPED_DATA_PATH,dtype={'formated_number': str})
#result_scraper.rename(columns={'Base Website': 'web'}, inplace=True)
result_scraper.head()

,Unnamed: 0,Base Website,Scraped Page,Page Type,Emails,Phone Numbers
0,0,https://www.fokuslabe.cz,https://www.fokuslabe.cz,main,"teplice@fokuslabe.czDocument, fokus@fokuslabe....","730 891 498, 603 863 071, 730 158 910, 731 449..."
1,1,https://www.fokuslabe.cz,https://www.fokuslabe.cz/kontakty,contact,"teplice@fokuslabe.czDocument, fokus@fokuslabe....","730 891 498, 733 184 388, 603 863 071, 472 745..."
2,2,https://www.fokuslabe.cz,https://www.fokuslabe.cz/o-nas,contact,NaN,NaN
3,3,https://www.animaviva.cz,https://www.animaviva.cz,main,"poradna@animaviva.cz, koordinator.zamozp@anima...","553 821 301, 737 500 030, 420776738, 739 404 5..."
4,4,https://www.animaviva.cz,https://www.animaviva.cz/kontakt,contact,"poradna@animaviva.cz, asistentka@animaviva.cz,...","553 821 301, 737 500 030, 739 404 544, 739 404..."


In [62]:

res_2 = pd.merge(res_unmatched, result_scraper,on = ["web"],how="inner")[["web", "E-mail", "Telefon", "Matched", "Phone Numbers"]].sort_values(by = ["web","Telefon"], ascending = [True, True])
res_2.to_csv("res_2.csv")

TypeError: to_csv() got an unexpected keyword argument 'dtype'

In [63]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm 
import phonenumbers
import phonenumbers.geocoder
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import datetime
import os

In [64]:
def py_parse_phonenumber(num):
    try:
        parsed_num = phonenumbers.parse(num, 'CZ')
        phonenumbers.is_possible_number_with_reason(parsed_num)
        return {
            'formated_number':phonenumbers.format_number(parsed_num, phonenumbers.PhoneNumberFormat.E164),
            'number': parsed_num.national_number,
            'prefix': parsed_num.country_code,
            'country_code': phonenumbers.region_code_for_number(parsed_num),
            'valid': phonenumbers.is_valid_number(parsed_num),
            'possible': phonenumbers.is_possible_number(parsed_num),
            'parsed': True
        }
    except Exception as e:
        return {'number': num, 'prefix': None, 'country_code': None, 'valid': False, 'possible': False, 'parsed': False}

def udf(df: pd.DataFrame, column_name: str):
    results = df[column_name].apply(py_parse_phonenumber)
    parsed_df = pd.DataFrame(results.tolist())
    return parsed_df

def explode_df(df:pd.DataFrame, column_name:str) -> pd.DataFrame:
    #df_res = pd.read_csv("../data/1_06/df_scraped_full.csv")
    df_res = df
    df_res[f'{column_name}_scraped'] = df_res[column_name].str.split(', ')
   # df_res['emails_scraped'] = df_res['Emails'].str.split(', ')
    df_res_phones = df_res[["Base Website", "Scraped Page",f"{column_name}_scraped"]]
    df_res_exp = df_res_phones.explode(f'{column_name}_scraped').reset_index(drop=True)
    return df_res_exp

def has_more_than_5_consecutive_zeros(number):
    return bool(re.search(r'0{4,}', str(number)))


def clean_scraped_phones(df: pd.DataFrame, phone_scraped_column = "Phone Numbers") -> pd.DataFrame:
    phones_exp = explode_df(df,phone_scraped_column)
    ress_df = udf(phones_exp,f"{phone_scraped_column}_scraped")
    phones_exp["formated_number"] = ress_df["formated_number"]
    phones_exp.drop(columns=[f"{phone_scraped_column}_scraped"], inplace= True)
    phones_deduped = phones_exp.drop_duplicates(subset=['Base Website', 'formated_number'])
    filtered_df = phones_deduped[~phones_deduped['formated_number'].apply(has_more_than_5_consecutive_zeros)]
    phones_df = filtered_df.sort_values(by="formated_number", ascending=True)
    return phones_df

In [73]:
SCRAPED_DATA_PATH = '/Users/dmitriisid/Desktop/python/webscraping/data/2_06/df_scraped_2_06.csv'
result_scraper = pd.read_csv(SCRAPED_DATA_PATH,dtype={'formated_number': str})
akademickyustav = result_scraper[result_scraper["Base Website"]=="http://www.akademickyustav.cz/oaza"]

In [81]:
phone_scraped_column = "Phone Numbers"
phones_exp = explode_df(akademickyustav,phone_scraped_column)
ress_df = udf(phones_exp,f"{phone_scraped_column}_scraped")
phones_exp["formated_number"] = ress_df["formated_number"]
phones_exp.drop(columns=[f"{phone_scraped_column}_scraped"], inplace= True)
print(phones_exp)
phones_deduped = phones_exp.drop_duplicates(subset=['Base Website', 'formated_number'])
filtered_df = phones_deduped[~phones_deduped['formated_number'].apply(has_more_than_5_consecutive_zeros)]
phones_df = filtered_df.sort_values(by="formated_number", ascending=True)
print(phones_df)

                         Base Website  \
0  http://www.akademickyustav.cz/oaza   
1  http://www.akademickyustav.cz/oaza   
2  http://www.akademickyustav.cz/oaza   
3  http://www.akademickyustav.cz/oaza   
4  http://www.akademickyustav.cz/oaza   
5  http://www.akademickyustav.cz/oaza   
6  http://www.akademickyustav.cz/oaza   

                                    Scraped Page formated_number  
0             http://www.akademickyustav.cz/oaza   +420731310286  
1             http://www.akademickyustav.cz/oaza   +420724112344  
2             http://www.akademickyustav.cz/oaza   +420731310286  
3    https://www.akademickyustav.cz/oaza/o-oaze/   +420731310286  
4  https://www.akademickyustav.cz/oaza/kontakty/   +420731310286  
5  https://www.akademickyustav.cz/oaza/kontakty/   +420724112344  
6  https://www.akademickyustav.cz/oaza/kontakty/   +420731310286  
                         Base Website                        Scraped Page  \
1  http://www.akademickyustav.cz/oaza  http://www.akademic

/var/folders/rd/l77bltt96_97jdz4bbkmq5z40000gn/T/ipykernel_39963/4153782861.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res[f'{column_name}_scraped'] = df_res[column_name].str.split(', ')


In [ ]:

df_phones_scraped = clean_scraped_phones(result_scraper)
df_phones_scraped.to_csv("df_phones_scraped.csv")